#### This file contains the code to:
1. Prepare data for the model
2. Define and train the model
3. Save and test the model


#### This file requires the following directories to be set up in advance:

1. Faces/1/

    ->Contains 1000+ images of the face you want to track
    
2. Faces/2/ 

    ->Contains 1000+ images of unwanted faces while tracking
    
3. Faces/1 Test/

    ->Test images of the face to track
    
4. Faces/2 Test/ 

    ->Test images of unwanted faces

*These directories aren't included as they contain sensitive data.*

In [1]:
import numpy as np
import cv2
import pickle
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

##### Loading and preparing data

In [ ]:
DATADIR = os.path.join(os.getcwd(),'Faces')
CATEGORIES = ["1","0"]

for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
        img_array = cv2.resize(img_array, (100,100))
        plt.imshow(img_array, cmap = "gray")
        plt.show()
        break

In [2]:
IMG_SIZE = 100

training_data = []

def create_training_data():
    """
    Populate training_data with [image_array,label] 
    """
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        label = CATEGORIES.index(category)
        for img in tqdm(os.listdir(path)):
            try:
                img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
                training_data.append([new_array,label])
            except Exception as e:
                pass

In [3]:
create_training_data()

100%|██████████| 1012/1012 [00:01<00:00, 542.66it/s]


In [4]:
print(len(training_data))
random.shuffle(training_data)

2110


In [5]:
X = []
y = []

#### Saving training data

In [6]:
for features,label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

pickle_out = open('X.pickle','wb')
pickle.dump(X,pickle_out)
pickle_out.close()

pickle_out = open('y.pickle','wb')
pickle.dump(y,pickle_out)
pickle_out.close()

#### Model creation, training and saving

In [7]:
X = pickle.load(open('X.pickle', 'rb'))
y = np.array(pickle.load(open('y.pickle', 'rb')))
X = X/255.0

#Tensorflow Conv2D Sequential Model for image classification
model = Sequential()
model.add(Conv2D(64,(3,3), input_shape = X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss = 'binary_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])
model.fit(X, y, batch_size = 32, validation_split = 0.1, epochs = 10)

Epoch 1/10
60/60 [==============================] - 40s 645ms/step - loss: 0.4695 - accuracy: 0.7948 - val_loss: 0.0687 - val_accuracy: 0.9668
Epoch 2/10
60/60 [==============================] - 28s 473ms/step - loss: 0.0614 - accuracy: 0.9761 - val_loss: 0.0310 - val_accuracy: 0.9905
Epoch 3/10
60/60 [==============================] - 30s 494ms/step - loss: 0.0385 - accuracy: 0.9902 - val_loss: 0.0323 - val_accuracy: 0.9810
Epoch 4/10
60/60 [==============================] - 29s 482ms/step - loss: 0.0349 - accuracy: 0.9852 - val_loss: 0.0193 - val_accuracy: 0.9953
Epoch 5/10
60/60 [==============================] - 29s 482ms/step - loss: 0.0141 - accuracy: 0.9962 - val_loss: 0.0077 - val_accuracy: 0.9953
Epoch 6/10
60/60 [==============================] - 28s 467ms/step - loss: 0.0074 - accuracy: 0.9972 - val_loss: 0.0210 - val_accuracy: 0.9905
Epoch 7/10
60/60 [==============================] - 28s 472ms/step - loss: 0.0036 - accuracy: 0.9999 - val_loss: 0.0157 - val_accuracy: 0.9905

In [8]:
model.save('Face-Recognition.model')

INFO:tensorflow:Assets written to: Face-Recognition.model/assets


## Testing the model
Separate files:
1. 1 Test = Face you want to detect
2. 2 Test = Faces you would ignore

In [1]:
CATEGORIES =['You','Not You']
names = ["1 Test", "2 Test"]
pathname = "Faces"

filepaths = []

for name in names:
    path = os.path.join(pathname,name)
    for file in os.listdir(path):
        filepath = os.path.join(path,file)
        filepaths.append([filepath,CATEGORIES[names.index(name)]])
    
def prepare(filepath):
    IMG_SIZE = 100
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))/255.0
    return  (np.array(new_array).reshape(-1,IMG_SIZE,IMG_SIZE,1).astype('float32'),img_array)

model = tf.keras.models.load_model("Face-Recognition.model")

for file in filepaths:
    prepare_out = prepare(file[0])
    prediction = model.predict(prepare_out[0])
    print(prediction, file[1])

[[0.50489295]] You
[[0.3065823]] You
[[0.7550906]] You
[[0.5193197]] You
[[0.11118051]] You
[[0.23189941]] You
[[0.5050461]] You
[[0.9251879]] You
[[0.5583346]] You
[[0.6208454]] You
[[0.5095783]] You
[[0.3886503]] You
[[0.6720265]] You
[[1.]] Not You
[[0.99733186]] Not You
[[0.9999994]] Not You
[[1.]] Not You
[[0.99999976]] Not You
[[1.]] Not You
[[1.]] Not You
[[1.]] Not You
[[1.]] Not You
[[1.]] Not You
[[1.]] Not You
[[1.]] Not You
[[1.]] Not You
[[0.999994]] Not You
[[1.]] Not You
[[0.98007053]] Not You
[[1.]] Not You
[[1.]] Not You
[[1.]] Not You
[[1.]] Not You
[[1.]] Not You
[[1.]] Not You
[[0.9999964]] Not You
